# Part 1: Spark and Spark SQL

## Task2 : Webgraph on Internal Links

In [1]:
## initial Setup

import re 
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, explode, lower
from pyspark.sql.types import StringType, ArrayType

spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/23 18:16:39 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/04/23 18:16:39 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/04/23 18:16:39 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/04/23 18:16:39 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [2]:
## df

small = spark.read.format('xml').options(rowTag='page').load('hdfs:/enwiki_small.xml')
test = spark.read.format('xml').options(rowTag='page').load('hdfs:/enwiki_test.xml')

In [5]:
## internal link filter function
def get_internal_link(text):
    matches = re.findall(r'\[\[(.*?)\]\]', text)
    for i, match in enumerate(matches):
        matches[i] = re.split('\|', match)[0]
       
    result = []
    for match in matches:
        if len(re.findall(r'^category:', match)) >0:
            result.append(match)
        elif len(re.findall(r':|#', match)) > 0:
            continue
        else:
            result.append(match)
    return result

get_internal_link = udf(get_internal_link, ArrayType(StringType()))

## map reduce function
def process(df):
    output = df.select(lower(df.title).alias('title'), lower(df.revision.text._Value).alias('text')) \
            .dropna(how = 'any') \
            .withColumn('internal_link', get_internal_link('text')) \
            .select('title', explode('internal_link').alias('internal_link')) \
            .sort('title', 'internal_link')
    return output

In [6]:
# small
small_result = process(small).limit(5)
small_result.show()

+----------------+--------------------+
|           title|       internal_link|
+----------------+--------------------+
|"love and theft"| mississippi (bob...|
|"love and theft"|                2001|
|"love and theft"|accidents & accus...|
|"love and theft"|           accordion|
|"love and theft"|            allmusic|
+----------------+--------------------+



In [7]:
# test
test_result = process(test).limit(5)
test_result.show()

+-----------+-------------------+
|      title|      internal_link|
+-----------+-------------------+
|  ! (chess)|punctuation (chess)|
|!!! (album)|                !!!|
|!!! (album)|                !!!|
|!!! (album)|              album|
|!!! (album)|              album|
+-----------+-------------------+



In [9]:
#save samll result to gcs
gcs_filepath = 'gs://hw2-backup/p1t2.csv'
small_result.coalesce(1).write.option("delimiter", "\t").csv(gcs_filepath)